In [1]:
data_dir = "test_data"
model = "full_model"

In [ ]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import statistics
from sklearn.linear_model import LinearRegression

# Set the font family and size for LaTeX rendering
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 10

plt.rcParams.update({"text.usetex": False})

# Set the font used for math expressions to LaTeX
plt.rcParams["mathtext.fontset"] = "cm"

In [3]:
# Read from file_paths.csv file. Each row corresponds to one model fit in stan. There should be three columns containing the following:
# 1. stan_sample_file: path to file where stan samples are saved
# 2. data_file: path to the data the model was fit on (this file should contain the true parameter values)

results = pd.read_csv(os.path.join('file_paths', data_dir + '_' + model + '_file_paths.csv'))

In [4]:
# Get all simulated data 
simulated_datas = {}
for file_name in results['data_file'].tolist():
    file = open(file_name, 'rb')
    simulated_datas[file_name] = pickle.load(file)
    file.close()

In [ ]:
true_vals = {}
posterior_vals = {}
true_vals_converging = {}
posterior_vals_converging = {}
rhat_vals = {}
variables = ['F_constrained[0]', 'F_unconstrained[0]', 'F_unconstrained[1]', 'F_unconstrained[2]',
                   'F_intercept[0]', 'F_intercept[1]', 'F_intercept[2]', 'F_intercept[3]',
                   'beta_a', 'beta_z', 'beta_0', 'mu_r[0]', 'mu_r[1]', 'sigma_r[0]', 'sigma_r[1]', 'mu_z0_1', 'sigma_z0_1', 
                   'sigma_eps[0]', 'sigma_eps[1]', 'sigma_eps[2]', 'sigma_eps[3]']

display_variables = [r'$F[0]$', 
                     r'$F[1]$', 
                     r'$F[2]$', 
                     r'$F[3]$',
                     r'$b[0]$', 
                     r'$b[1]$', 
                     r'$b[2]$', 
                     r'$b[3]$',
                     r'$\beta_A^{(1)}$', 
                     r'$\beta_Z$', 
                     r'$\beta_0$', 
                     r'$\mu_R^{(0)}$', 
                     r'$\mu_R^{(1)}$', 
                     r'$\sigma_R^{(0)}$', 
                     r'$\sigma_R^{(1)}$', 
                     r'$\mu_{Z_0}^{(1)}$', 
                     r'$\sigma_{Z_0}^{(1)}$', 
                     r'$\psi[0]$', 
                     r'$\psi[1]$', 
                     r'$\psi[2]$', 
                     r'$\psi[3]$']

for v in display_variables:
    true_vals[v] = []
    posterior_vals[v] = []
    rhat_vals[v] = []
    true_vals_converging[v] = []
    posterior_vals_converging[v] = []

total_runs = 0

for j in range(len(results)):
    file = results.iloc[j]['stan_sample_file']
    if not os.path.isfile(file):
        continue
    total_runs += 1
    
    df = pd.read_csv(file)
    latent_params = simulated_datas[results.iloc[j]['data_file']]['latent_params']
    observed_data = simulated_datas[results.iloc[j]['data_file']]['observed_data']
    
    true_vals[r'$F[0]$'].append(latent_params['F'][0])
    true_vals[r'$F[1]$'].append(latent_params['F'][1])
    true_vals[r'$F[2]$'].append(latent_params['F'][2])
    true_vals[r'$F[3]$'].append(latent_params['F'][3])
    true_vals[r'$b[0]$'].append(latent_params['F_intercept'][0])
    true_vals[r'$b[1]$'].append(latent_params['F_intercept'][1])
    true_vals[r'$b[2]$'].append(latent_params['F_intercept'][2])
    true_vals[r'$b[3]$'].append(latent_params['F_intercept'][3])
    true_vals[r'$\beta_A^{(1)}$'].append(latent_params['beta_a'])
    true_vals[r'$\beta_Z$'].append(latent_params['beta_z'])
    true_vals[r'$\beta_0$'].append(latent_params['beta_0'])
    true_vals[r'$\mu_R^{(0)}$'].append(latent_params['mu_r'][0])
    true_vals[r'$\mu_R^{(1)}$'].append(latent_params['mu_r'][1])
    true_vals[r'$\sigma_R^{(0)}$'].append(latent_params['sigma_r'][0])
    true_vals[r'$\sigma_R^{(1)}$'].append(latent_params['sigma_r'][1])
    true_vals[r'$\mu_{Z_0}^{(1)}$'].append(latent_params['mu_z0'][1])
    true_vals[r'$\sigma_{Z_0}^{(1)}$'].append(latent_params['sigma_z0'][1])
    true_vals[r'$\psi[0]$'].append(latent_params['sigma_eps'][0])
    true_vals[r'$\psi[1]$'].append(latent_params['sigma_eps'][1])
    true_vals[r'$\psi[2]$'].append(latent_params['sigma_eps'][2])
    true_vals[r'$\psi[3]$'].append(latent_params['sigma_eps'][3])
    
    for i in range(len(variables)):
        var = variables[i]
        display_var = display_variables[i]
        posterior_vals[display_var].append(df[df['Unnamed: 0'] == var]['mean'].iloc[0])
        rhat_vals[display_var].append((j, df[df['Unnamed: 0'] == var]['r_hat'].iloc[0]))

min_pearson_r = 1
max_pearson_r = 0
sum_pearson_r = 0
sum_slopes = 0
pearson_r_list = []
slopes_list = []

figure = plt.figure(figsize=(8,10))

figure.text(0.52, 1, 'Model parameter recovery', ha='center', fontsize=16)
figure.text(0.5, 0, 'Inferred values', ha='center', fontsize=14)
figure.text(0, 0.5, 'True values', va='center', rotation='vertical', fontsize=14)

for i in range(len(display_variables)): 
    v = display_variables[i]
    ax = figure.add_subplot(5, 5, i+1)
    for j in range(len(true_vals[v])):
        if rhat_vals[v][j][1] < 1.1:
            true_vals_converging[v].append(true_vals[v][j])
            posterior_vals_converging[v].append(posterior_vals[v][j])
    ax.scatter(posterior_vals[v], true_vals[v], s=8, c='blue', alpha=0.5)
    
    pearson_r = scipy.stats.pearsonr(true_vals[v], posterior_vals[v])
    sum_pearson_r += pearson_r.statistic
    pearson_r_list.append(pearson_r.statistic)
    
    model = LinearRegression(fit_intercept=False)
    slope = model.fit(np.array(posterior_vals[v]).reshape(-1, 1), np.array(true_vals[v]).reshape(-1, 1)).coef_[0]
    sum_slopes += slope
    slopes_list.append(slope)
    
    plt.xlim(min(true_vals[v] + posterior_vals[v]), max(true_vals[v] + posterior_vals[v]))
    plt.ylim(min(true_vals[v] + posterior_vals[v]), max(true_vals[v] + posterior_vals[v]))
    plt.text(0.5, 1.33, v, ha='center', va='center', transform=ax.transAxes, fontsize=14)
    plt.text(0.5, 1.1,  'R=' + str(round(pearson_r.statistic, 2)), ha='center', va='center', transform=ax.transAxes, fontsize=10, color='gray')
    ax.axline((0,0), slope=1, color='black', linestyle='--')
    ax.axis('square')

print('Total # finished runs:', total_runs)
print()
print('Parameter recovery statistics:')
print('Mean correlation:', sum_pearson_r / len(display_variables))
print('Mean slope:', sum_slopes / len(display_variables))
print('Median correlation:', statistics.median(pearson_r_list))
print('Median slope:', statistics.median(slopes_list))

figure.tight_layout()
plt.savefig(os.path.join('figures', 'param_recovery.pdf'), bbox_inches='tight')
plt.show()